<a href="https://colab.research.google.com/github/GisstHubana/FHIR_Reporting_Tool/blob/GisstHubana-patch-html/Code/fhir_reporting_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung
## CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books für eine bessere Reproduzierbarkeit

In [ ]:
%reset -f

## Installieren notwendiger Pakete

In [ ]:
!pip install pyngrok
!pip install flask-ngrok

## gnrok mit Authorizierungstoken verknüpfen

In [ ]:
!ngrok authtoken 2Wlt2vGzytTpJ6zABKGKKwfiTof_7y4sewzGkNGy7zW6GB5oz

Importieren der notwendigen Libraries

In [ ]:
from flask import Flask, request
from flask.templating import render_template
from flask_ngrok import run_with_ngrok
import secrets
import sqlite3

Erstellung einer virtuellen Kopie des GitHub Repositorium FHIR_Reporting_Tool in Colab

In [ ]:
%cd /content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde

!git clone https://github.com/GisstHubana/FHIR_Reporting_Tool.git #virtuelle Kopie des Repositoriums - Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
!ls # Auflistung aller Ordnerinhalte

### Database in Flask

#### Use database to match login credentials

In [ ]:
import sqlite3
db_folder = "/content/FHIR_Reporting_Tool/"
conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
cursor = conn.cursor()

#query
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT NOT NULL,
  password TEXT NOT NULL

)

"""
cursor.execute(create_table_query)

insert_query = """
INSERT INTO users(username, password)
SELECT ?, ?
WHERE NOT EXISTS (
  SELECT 1 FROM users WHERE username = ?

)
"""
user_data = ("USER3", "1234", "USER3")
cursor.execute(insert_query, user_data)
conn.commit()

select_query = "SELECT * FROM users"
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)


In [ ]:
template_folder = "/content/FHIR_Reporting_Tool/templates"
static_folder = "/content/FHIR_Reporting_Tool/static"
db_folder = "/content/FHIR_Reporting_Tool/"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
# app.debug = True # Debug-Modus aktivieren - webseite lädt nicht wenn eingeschaltet
app.secret_key = secrets.token_hex(32)
run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

@app.route('/', methods=["GET", "POST"])
def login():

  if request.method == "POST":
    username = request.form.get("username")
    password = request.form.get("password")

    conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
    cursor = conn.cursor()
    # query
    query = """
    SELECT * FROM users WHERE username = ? AND password = ?
    """
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
     return render_template("page2.html", name=username)
    else:
      message = "Wrong username or password. Please try again!"
      return render_template("login.html", message=message)

  else:
    return render_template("login.html")

@app.route('/home') # Erstellung einer Routendefinition home
def index():
    return render_template("index.html")

@app.route('/page2')
def page2():
    username = input("Enter your name:")
    return render_template("page2.html", name=username)

@app.route('/page3')
def page3():
    return render_template("page3.html")

app.run()

## Abschluss - Runtime löschen und abschließend clear all outputs

In [ ]:
# Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
from google.colab import runtime
runtime.unassign() # delete runtime